# Molecular dynamics simulation of a small protein using GROMACS 

    authors  : Alessandra Villa (based on a Justin Lemkuhl tutorial see http://www.mdtutorials.com or Living J. Comp. Mol. Sci. 2018, 1, 5068).    
    goal     : learn step-by-step how to run a molecular dynamics simulation of a small protein using GROMACS 
    time     : 90 minutes
    software : GROMACS 2023, python modules: numpy, matplotlib, re, nglviewer, md_traj, panda. 
    optional software: visualization software [VMD](https://www.ks.uiuc.edu/Research/vmd), Xmgrace plotting tool
    tutorial source: tutorials.gromacs.org
    version  : release
    website  : https://tutorials.gromacs.org/docs/md-intro-tutorial.html

# Preparations 

In [ ]:
# Change to the data directory
%cd data

### Obtaining the input for a simulation

For this tutorial, we will utilize Factor Xa, a protein playing critical role in the formation of blood clots. 
- The 3D structure is available from the RCSB website, https://www.rcsb.org/ with PDB code 1FJS. 
- You can find the PDB file for the crystal structure in "input" directory as "1fjs.pdb".

Visualizing the structue: both approaches to visualization did not work for me, but leaving them here for review. 
- nglview 
- vmd 


In [ ]:
# nglview
# %pip install nglview
import nglview as ng
view = ng.show_structure_file("input/1fjs.pdb")
view
# click and drag to rotate, zoom with your mouseweel 
# for more infor on this viewer have a look at https://github.com/nglviewer/nglview

In [ ]:
# vmd
!vmd input/1fjs.pdb

### Cleaning the input structure

Strip out all the atoms that do not belong to the protein (e.i crystal waters, ligands, etc). Atoms (labelled "HETATM" in the PDB file) and eventually their connectivity. In other words, verify that all the necessary atoms are present, and the PDB file contains only protein atoms.
- Use a plain text editor like vi, emacs (Linux/Mac), or Notepad (Windows). 
- Do not use word processing software! 
- Alternatively use grep to delete these lines very easily

In [6]:
# remove lines with HETATM and CONECT from the pdb file
!grep -v HETATM input/1fjs.pdb > 1fjs_protein_tmp.pdb
!grep -v CONECT 1fjs_protein_tmp.pdb > 1fjs_protein.pdb

In [ ]:
# Visualize the cleaned structure (optional)
# import nglview as ng
# view = ng.show_structure_file("1fjs_protein.pdb")
# view
# click and drag to rotate, zoom with your mouseweel 
# for more infor on this viewer have a look at https://github.com/nglviewer/nglview

# Alternative approach below 
#!vmd 1fjs_protein.pdb

Check your .pdb file for entries listed under the comment MISSING, as these entries indicate either atoms or whole residues that are not present in the crystal structure. 
- Terminal regions may be absent, and may not present a problem for dynamics.

In [ ]:
# check for MISSING comment in pdb file  
!grep MISSING input/1fjs.pdb

### Generating a topology

PDB file is ready to be input into GROMACS 
We use [`gmx pdb2gmx`](https://manual.gromacs.org/current/onlinehelp/gmx-pdb2gmx.html) to generate three files:

* The topology for the molecule.
* A position restraint file.
* A post-processed structure file. 

The topology (**topol.top** by default) contains all the information necessary to define the molecule within a simulation.
- nonbonded parameters (atom types and charges)
- bonded parameters (bonds, angles, dihedrals and atom connectivity)

The purpose of `gmx pdb2gmx` is to produce a force field-compliant topology

Note: 
- *Incomplete internal sequences or any amino acid residues that have missing atoms will cause `gmx pdb2gmx` to fail.
- missing atoms/residues must be modeled in using other software packages. Also note that 
- `gmx pdb2gmx` is not magic, it cannot generate topologies for arbitrary molecules, just the residues defined by the force field (in the *.rtp files - generally proteins, nucleic acids, and a very finite amount of cofactors, like NAD(H) and ATP).*

In [10]:
# create the three files using gmx pdb2gmx
!gmx pdb2gmx -f 1fjs_protein.pdb -o 1fjs_processed.gro -water tip3p -ff "charmm27"

             :-) GROMACS - gmx pdb2gmx, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            

Choosing the CHARMM27 all-atom force field is an important decision.
- force field will contain the information that will be written to the topology.
- always read thoroughly about each force field and decide which is most applicable to your situation. 
- when running `gmx pdb2gmx` without the -ff flag, choices are given, additional command line input is required. 

There are many other options that can be passed to `gmx pdb2gmx` (see http://manual.gromacs.org/documentation/current/onlinehelp/gmx-pdb2gmx.html). 

We have now generated three new files: 
- **1fjs_processed.gro**:  GROMACS-formatted structure file that contains all the atoms defined within the force field (i.e., H atoms have been added to the amino acids in the protein).
- **topol.top**: system topology
The posre files contain information used to restrain the positions of heavy atoms.
- **topol_Protein_chain_X.itp** 
- **posre_Protein_chain_X.itp**


### Understanding molecule "topologies"

Using a plain text editor, you can look at what is in the output topology (topol.top)
- `!cat topol.top`
- "Protein_chain_A" defines the molecule name, based on the fact that the protein was labeled as chain A in the PDB file. 
- There are 3 exclusions for bonded neighbors. More information on exclusions <a href=http://manual.gromacs.org/current/reference-manual/topologies.html>GROMACS manual</a>.

#### Atoms in a topology

The next section defines the [ atoms ] in the protein. 
- `! grep "atoms" -A 4 topol_Protein_chain_A.itp`

|Field  |  description |
|--|--
|nr|Atom number
|type| Atom type
|resnr|Amino acid residue number
|residue| The amino acid residue name- Note that this may be different from .rtp entry. 
|atom| Atom name
|cgnr| Charge group number - Not used anymore 
|charge| Self-explanatory - The "qtot" descriptor is a running total of the charge on the molecule
|mass| Also self-explanatory
|typeB, chargeB, massB| Used for free energy perturbation (not discussed here) 

#### Bonds and other interactions in a topology

Subsequent sections include [ bonds ], [ pairs ], [ angles ], and [ dihedrals ]. Some of these sections are self-explanatory (bonds, angles, and dihedrals. 
- `!grep "bonds" -A 2 topol_Protein_chain_A.itp`
- `!grep "pairs" -A 2  topol_Protein_chain_A.itp`
- `!grep "angles" -A 2 topol_Protein_chain_A.itp`
- `!grep "dihedrals" -A 2 topol_Protein_chain_A.itp`
- interactions are found [here](http://manual.gromacs.org/current/reference-manual/functions.html) 
special 1-4 interactions are included under [pairs](http://manual.gromacs.org/current/reference-manual/functions/interaction-methods.html#exclusions-and-1-4-interactions). 
- format associated to function types [see topology file table](http://manual.gromacs.org/documentation/2020-beta1/reference-manual/topologies/topology-file-formats.html) in GROMACS manual.

#### Water and position restraints

- `!grep "posre" topol*.itp`
- the "posre.itp" file defines a force constant used to keep atoms in place during equilibration.
- use the following command to view it `!cat posre_Protein_chain_A.itp`
- The remainder of the topology file is dedicated to defining other molecules and providing system-level descriptions. 
- The next moleculetype (by default) is the solvent, in this case TIP3P **water**. For an excellent summary of the many different water models, click [here](http://www1.lsbu.ac.uk/water/water_models.html).

#### Ions and other parameters

- !grep "ions" topol.top

#### System level definitions

- `!tail -8 topol.top`
- [ system ] directive gives the name of the system that will be written to output files during the simulation.
- [ molecules ] directive lists all of the molecules in the system.
- - The order of the listed molecules must exactly match the order of the molecules in the coordinate (in this case, .gro) file.
- - The names listed must match the `[ moleculetype ]` name for each species, not residue names or anything else.
- - Otherwise, you will get fatal errors from grompp about mismatched names, molecules not being found, or a number of others.

### Solvating the simulation system

In this example, we are going to be simulating a simple aqueous system. It is possible to simulate proteins and other molecules in different solvents, provided that good parameters are available for all species involved.

- Define the box dimensions using the [`gmx editconf`](https://manual.gromacs.org/current/onlinehelp/gmx-editconf.html) tool.
- Fill the box with water using the [`gmx solvate`](https://manual.gromacs.org/current/onlinehelp/gmx-solvate.html) tool. 

You are now presented with a choice as to how to treat the unit cell. For the purpose of this tutorial, we will use the rhombic dodecahedron

### Defining the simulation box

In [ ]:
# define the box: gmx editconf
!gmx editconf -f 1fjs_processed.gro -o 1fjs_newbox.gro -c -d 1.0 -bt dodecahedron

- `-c` center the protein in the box
- `-d 1.0` place it at least 1.0 nm from the box edge
- `-bt dodecahedron` box type is defined as a rhombic dodecahedron 
- For more on periodic boundary conditions and dodecahedrons [here](http://manual.gromacs.org/current/reference-manual/algorithms/periodic-boundary-conditions.html#pbc) 

### Filling the box with water

`gmx solvate` will keep track of how many water molecules it adds in this step, which it then writes to your topology to reflect the changes.

In [ ]:
# fill the box with water: gmx solvate 
!gmx solvate -cp 1fjs_newbox.gro -cs spc216.gro -o 1fjs_solv.gro -p topol.top

- `-cp` configuration of the proteinis contained in the output of `gmx editconf` step, 
- `-cs` configuration of the solvent (spc216.gro) is a generic equilibrated 3-point solvent model box.
- The output is called `1fjs_solv.gro`, and we tell solvate the name of the topology file (topol.top) so it can be modified. 

### Adding Ions

We now have a solvated system that contains a charged protein. 
- The output of `pdb2gmx` told us that the protein has a net charge of -2e (based on its amino acid composition). 
- -  look at the last line of each [ atoms ] directive in topology file; it should read  "qtot 1." for chain A and  "qtot -3." for chain L. 
- - Since life does not exist at a net charge, we must add ions to our system. 
- We aim to approximate physiological conditions and use therefore a NaCl concentration of 0.15 M.

### Preparing the input for "gmx genion"

The tool for adding ions within GROMACS is called [`gmx genion`](https://manual.gromacs.org/current/onlinehelp/gmx-genion.html). 
- `gmx genion` reads through the topology and replaces water molecules with the ions that the user specifies. 
  - `-s`: structure/state file
  - `-o`: output is `.gro` file
  - `-p`: process the topology to reflect the removal of water molecules and addition of ions. 
  - `-pname`, `-nname` define positive and negative ion names. (standardized, do not depend on the force field.)
  - `-neutral` tells the gmx genion command to add ions necessary to neutralize the net charge on the protein by adding the correct number of negative ions. 
  - `conc`: add a specified concentration of ions. works in conjunction with `-neutral`: to neutralize the system 
  - *Note: Make sure to run `gmx genion` only once. This edits the topology "in-place" and any additions to it will stack. Start over with the `pdb2gmx` if you accidentally run it twice.
- `.tpr` input file is called a run input file, produced by the GROMACS grompp module (GROMACS pre-processor), and contains all the parameters for all of the stoms in the system.
- `grompp` processes coordinate file + topology information + `.mdp` parameters to generate an atomic-level input (`.tpr`). 
  - `.mdp`: (molecular dynamics parameter file)

In [ ]:
# assembling / processing .tpr file with gmx grompp 
!gmx grompp -f ions.mdp -c 1fjs_solv.gro -p topol.top -o ions.tpr

*Note: A lot of GROMACS tools promt for input at the command line. As this does not play well with the jupyter notebook setup, we feed the input to the command line with a print command and a newline \n substituting enter*

In [ ]:
!printf "SOL\n" | gmx genion -s ions.tpr -o 1fjs_solv_ions.gro -conc 0.15 -p \
topol.top -pname NA -nname CL -neutral

We chose group "SOL" for embedding ions. You do not want to replace parts of your protein with ions.

### Energy minimization - Ensure that we have a reasonable starting structure in terms of geometry and solvent orientation

First we must ensure the system has no steric clashes or inappropriate geometry. Then the structure is relaxed through a process called energy minimization (EM).
- once again going to use `gmx grompp` to assemble the structure, topology, and simulation parameters into a binary input file (.tpr)
  - Its important that we have been updating our topol.top file when running genbox and genion, or else we will get lots of error messages ("number of coordinates in coordinate file does not match topology," etc).
- then use GROMACS MD engine, mdrun, to run the energy minimization
- `em.gro`: output file containing the energy minimized structure 
- `em.log`: ASCII-text log file containing more information on the run
- `em.edr`: file for storage of energy 
- `em.trr`: Binary full-precision trajectory file


In [ ]:
# prepare the .tpr binary input file
!gmx grompp -f input/emin-charmm.mdp -c 1fjs_solv_ions.gro -p topol.top -o em.tpr

In [ ]:
# run mdrun to carry out the energy minimisation:
!gmx mdrun -v -deffnm em

- `-v` makes `gmx mdrun` verbose such that it prints its progress to the screen at every step. 
  - Never use this flag if run in background, on a HPC center or on a local cluster, it prints unnecessary data in the standard output file. 
- `-deffnm` define the file names of the input and output. 
  - if you did not name your grompp output "em.tpr," you will have to explicitly specify its name with the `gmx mdrun` `-s` flag.

### Determining if the run was successful

two very important factors to evaluate  
- the potential energy: `Epot` should be negative and (for a simple protein in water) on the order of 100000 kJ/mol
- the maximum force: `Fmax`, the target for which was set in minim.mdp - `emtol = 1000.0` - indicating a target Fmax of no greater than 1000 kJ/(mol nm). 
  - It is possible to arrive at a reasonable Epot with Fmax > emtol. If this happens, your system may not be stable enough for simulation. 
  - Evaluate why it may be happening, and perhaps change your minimization parameters (integrator, emstep, etc).



### Analyzing the run results

- Analysis using `em.edr` file containing all of the energy terms.
- To analyse or visualize simulation data in python or jupyter notebooks, we can output a simplified xvg format from gmx-analysis tools with the option `-xvg none`

In [ ]:
# extract potential energy to a .xvg file 
!printf "Potential\n0\n" | gmx energy -f em.edr -o potential.xvg -xvg none

In [ ]:
# plot energy minimization curve
import pandas as pd
import plotly.express as px 

df = pd.read_csv('potential.xvg', sep='\s+', header=None, names=['step','energy'])
px.line(df, x='step', y='energy', title='Energy Minimization')


- EM ensures that we have a reasonable starting structurein terms of geometry and solvent orientation
- To begin real dynamics, we must equilibrate the solvent and ions around the protein.
  - If we were to attempt unrestrained dynamics at this point, the system may collapse because the solvent is mostly optimized within itself, and not necessarily with the solute, and ions are randomply placed by replacing water molecules. 

### Position restraints 

Use `posre.itp` to apply a position restraining force on the heavy atoms of the protein (anything that is not a hydrogen). 
- Movement is permitted, but only after overcoming a substantial energy penalty. 
- The utility of position restraints is that they allow us to relax our solvent and ions around our protein, without the added variable of structural changes in the protein. 
- The **origin** (the coordinates at which the restraint potential is zero) is provided via a coordinate file passed to the `-r` option of `grompp`. 

To use Positional Restraints, add \"define = -DPOSRES\" to the `.mdp` file 
- a file with restraint coordinates must be supplied with `-r` to `gmx grompp`. (can be the same file as supplied for `-c`)

### Equilibration run - temperature - stabilized the temperature of the system

The system needs to be brought to the temperature we wish to simulate and establish the proper orientation about the solute (the protein).
- After we arrive at the correct temperature (based on kinetic energies), we will apply pressure to the system until it reaches the proper density.

Equilibration is often conducted in two phases. 
- first phase is conducted under an NVT ensemble (constant Number of particles, Volume, and Temperature) AKA a "isothermal-isochoric" or "canonical." 
  - The timeframe for such a procedure is depends on the contents of the system. 100-200 ps should suffice
  - This may take a while.

Call `grompp` and `mdrun` just as we did at the EM step, except: 
- use energy minimised structure as input and a different `.mdp `file for the run.
- Instead of energy tolerance, we now give time a step size and a number of steps.
- Furthermore, we need to set a temperature.

Take note of a few parameters in the new `nvt-charmm.mdp` file:
- `gen_vel = yes`: Initiates velocity generation. Using different random seeds (gen_seed) gives different initial velocities, and thus multiple (different) simulations can be conducted from the same starting structure.
- `tcoupl = V-rescale`: The velocity rescaling thermostat is an improvement upon the Berendsen weak coupling method, which did not reproduce a correct kinetic ensemble.
- `pcoupl = no`: Pressure coupling is not applied. 
- A full explanation of the parameters used can be found in the GROMACS [manual](http://manual.gromacs.org/documentation/current/user-guide/mdp-options.html)

In [ ]:
# MD Equilibrium run - temperature. 
!gmx grompp -f input/nvt-charmm.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr 
!gmx mdrun -ntmpi 1 -v -deffnm nvt

Analyze the temperature progression, again using gmx energy:

In [ ]:
# extract temperature data
!echo "Temperature" | gmx energy -f nvt.edr -o temperature.xvg -xvg none -b 20

In [ ]:
# Plot temperature data 
import pandas as pd
import plotly.express as px 

df = pd.read_csv('temperature.xvg', sep='\s+', header=None, names=['step','temperature'])
px.line(df, x='step', y='temperature', title='Temperature Equilibration')

- Observe the plot and determine the temperature that the system reaches and remains stable at (target value (300 K))
- Determine an equilibrium period that may be adequate for this system (ps).

### Equilibration run - pressure - stabilize the pressure (and thus also the density) of the system

- Equilibration of pressure is conducted under an NPT ensemble, wherein the Number of particles, Pressure, and Temperature are all constant. 
- The ensemble is also called the "isothermal-isobaric" ensemble, and most closely resembles experimental conditions.

A new `npt-charmm.mdp` file will be used. 
- Note is not drastically different from the parameter file used for NVT equilibration, note the addition of the pressure coupling section. 

A few other changes:
- `continuation = yes`: Confirms we are continuing the simulation from the NVT equilibration phase
- `gen_vel = no`: Velocities are read from the trajectory (see below) 

Call `grompp` and `mdrun` just as we did for temperature equilibration.
- include `-t` flag to include the checkpoint file from the nvt equilibration
  - this file contains all the necessary state variables to continue our simulation. 
- to conserve the velocities produced during nvt, we must include the final coordinate file (output) of the nvt simulation using `-c` option

In [ ]:
# update the files using grompp
!gmx grompp -f input/npt-charmm.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

In [ ]:
# start an MD simulation just like before
!gmx mdrun -ntmpi 1 -v -deffnm npt

Analyze the pressure progression, again using energy:

In [ ]:
# extract pressure data
!echo "Pressure" | gmx energy -f npt.edr -o pressure.xvg -xvg none

In [ ]:
# Plot the pressure data 
import pandas as pd
df = pd.read_csv('pressure.xvg', sep='\s+', header=None, names=['step','pressure'])
px.line(df, x='step', y='pressure', title='Pressure Equilibration')

Pressure is a quantity that fluctuates widely over the course of an MD simulation
- as is clear from the large root-mean-square fluctuation (in the order of 100 bar)
- statistically speaking, one cannot distinguish a difference between the obtained average and the target/reference value (1 bar).

Let's take a look at density:

In [ ]:
# extract density data 
!echo "Density" | gmx energy -f npt.edr -o density.xvg -xvg none

In [ ]:
# plot density data 
import pandas as pd
df = pd.read_csv('density.xvg', sep='\s+', header=None, names=['step','density'])
px.line(df, x='step', y='density', title='Density Equilibration')

To determine if the system is well-equilibrated now with respect to pressure and density: 
- observe the densiry plot to see if density values are very stable over time

Note: Pressure-related terms are slow to converge, you may have to run NPT equilibration slightly longer than is specified here.

### The "production" run

We are now ready to release the position restraints and run production MD for data collection. 
- We will make use of the checkpoint file (which in this case now contains preserve pressure coupling information) to `grompp`. We will run a 1-ns MD simulation. 
  - Note we have explictly add a section that controls the output frequency in log file (`.log`), energy file (`.edr`), in the trajcotry file (`.trr`) and the compress trajectory file (`.xtc`).
  - note we will be using a new mdp file `md-charmm.mdp`

Here, we use velocity-rescaling temperature coupling as thermostat and stochastic cell rescaling as barostat. 
- A full explanation of the available thermostats and barostats in GROMACS can be found in the manual (see [here](http://manual.gromacs.org/documentation/current/reference-manual/algorithms/molecular-dynamics.html#temperature-coupling) for thermostat and [here](http://manual.gromacs.org/documentation/current/reference-manual/algorithms/molecular-dynamics.html#pressure-coupling) for barostat). 

Note, for thermostats and barostats we need to compute temperature or pressure. This requires global communication and is currently not done on GPU. To reduce these computational cost, 
  - [nsttcouple](https://manual.gromacs.org/current/user-guide/mdp-options.html#mdp-nsttcouple) frequency for coupling temperature set to 100 by default
  - [nstpcouple](https://manual.gromacs.org/current/user-guide/mdp-options.html#mdp-nstpcouple) frequency for coupling pressure set to 100 by default
  - Recommended to use `tau_t = 1` ps for V-rescale 
  - Recommended to use `tau_p = 5` ps for C-rescale

`gmx grompp` will print an estimate for generated data and PME load. 
- PME load will dictate how many processors should be dedicated to the PME calculation, and how many for the PP (everything except for PME) calculations. [manual](http://manual.gromacs.org/documentation/current/user-guide/mdrun-performance.html#parallelization-schemes)

In [ ]:
# the produciton run `grompp`
!gmx grompp -f input/md-charmm.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr

In [ ]:
# The production run `mdrun`
!gmx mdrun -ntmpi 1 -v -deffnm md

### Analysis

What types of data are important? Wha tools do we have for analysis?
- [`gmx trjconv`](https://manual.gromacs.org/current/onlinehelp/gmx-trjconv.html) which is used as a post-processing tool to strip out coordinates, correct for periodicity, or manually alter the trajectory (time units, frame frequency, etc)
  - we can use this to account for any periodicity in the system. 
  - `-center`: center atoms in the box 
  - options to specify input files: `-f`, `-s`
  - `-pcb`: sets the type of periodic boundary condition treatment. `-mol` puts the center of mass of molecules in the box, and requires a run input file to be supplied with `-s`.
- [`gmx mindist`](https://manual.gromacs.org/current/onlinehelp/gmx-mindist.html) 
  - use this to calculate the distance between the protein and its periodic image, to check that they do not interact with one another. 
  - The distance between the protein and its periodic image should not be smaller than the cut-off used to describe non-bonded interactions.
  - `-od`: output file
  - `-pi`: plot the minimum distance of a group to its periodic image. 
- [`gmx rms`](https://manual.gromacs.org/current/onlinehelp/gmx-rms.html) to check for structural stability 
  - calculate RMSD relative to the crystal structure present in the original pdb file 
  - `-tu`: unit for time value 
  - Choose 4 ("Backbone") for both the least-squares fit and the group for RMSD calculation. 

In [ ]:
# convert trajectory files 
!printf "1\n1\n" | gmx trjconv -s md.tpr -f md.xtc -o md_center.xtc -center -pbc mol

In [ ]:
# check distance between protein and its periodic image 
!printf "1\n" | gmx mindist -s md.tpr -f md_center.xtc -pi -od mindist.xvg 
#!xmgrace mindist.xvg

In [ ]:
# check Extract rmsd data
!printf "4\n1\n" | gmx rms -s em.tpr -f md_center.xtc -o rmsd_xray.xvg -tu ns -xvg none

In [ ]:
# plot rmsd
import pandas as pd
df = pd.read_csv('rmsd_xray.xvg', sep='\s+', header=None, names=['time','RMSD'])
df.plot('time')

### Radius of gyration

This is a measure of its compactness. 
- If a protein is stably folded, it will likely maintain a relatively steady value of Rg. 
- If a protein unfolds, its Rg will change over time. 
- Analyze the radius of gyration using GROMACS [`gmx gyrate`](https://manual.gromacs.org/current/onlinehelp/gmx-gyrate.html) 

In [ ]:
# analyze the radius of gyration
!echo "1" | gmx gyrate -f md_center.xtc -s md.tpr -o gyrate.xvg -xvg none

In [ ]:
# plot radius of gyration
import pandas as pd
df = pd.read_csv('gyrate.xvg', sep='\s+', header=None, names=['time','Rg'], usecols=[0, 1])
df.plot('time')

Stable protein in its compact (folded) form implies reasonably invariant Rg values. 

### Index file

Index groups are necessary for almost every GROMACS tools. 
- If one needs special index groups, he/she can use gmx [gmx make_ndx](https://manual.gromacs.org/current/onlinehelp/gmx-make_ndx.html) to generate an index file (ndx). 
  - For example the command `splitch 1` splits the group 1 (Protein) in chains and the command `q`  close the tool.  

In [ ]:
# generate index group
!printf "splitch 1\nq\n" | gmx make_ndx -f nvt.tpr -o 

Calculate the hydrogen bonds between the two protein chains using [gmx hbond](https://manual.gromacs.org/current/onlinehelp/gmx-hbond.html). 
  - The option `-num` provide the number of hydrogen bond as a function of time.  

In [ ]:
# Calculate hydrogen bonds 
!printf "17\n18\n"| gmx hbond -f md.xtc -s md.tpr  -n index.ndx -num -xvg none

In [ ]:
# plot bonds 
import pandas as pd
df = pd.read_csv('hbnum.xvg', sep='\s+', header=None, names=['time','H-bonds'], usecols=[0, 1])
df.plot('time')

### Generate Report 

Report or recall about what type of simulation we have performed.(`gmx report-methods`)[https://manual.gromacs.org/current/onlinehelp/gmx-report-methods.
html] can be useful for this purpose. 
- report methods print out basic system information on a performed run. 
- they also provide an unformatted text (with the option `-o`) or a LaTeX formatted output file with the option `-m`. 

In [ ]:
# generate report 
!gmx report-methods -s md.tpr